# TODO

TODO - finding back paintings informations

In [2]:
from selenium.webdriver.common.by import By
import pyautogui
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import os
import time
import splitfolders
import glob
import random
import shutil
from tqdm import tqdm
import pandas as pd

In [3]:
N_BATCHES = 10

def divide_in_batches():
    all_illus = glob.glob("illustrations/*.jpg")
    batch_len = int(len(all_illus)/N_BATCHES)
    random.seed(0)
    random.shuffle(all_illus)
    
    for i in tqdm(range(N_BATCHES)):
        for illu in all_illus[i*batch_len:(i+1)*batch_len]:
            dest = './batch' + str(i) + '/'
            shutil.copy(illu, dest)

#divide_in_batches()

In [16]:
def get_painting_info(driver, illu_id):
    title = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/div/div[4]/div/div/h1").get_attribute("innerHTML")
    specs = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/div/div[4]/div/div/p").get_attribute("innerHTML")
    author = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/div/div[4]/a/div/h3").get_attribute("innerHTML")
    try:
        link = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/div/div[4]/div[3]/a").get_attribute("href")
    except:
        link = ''
    return dict({'id': illu_id, 'title': title, 'specs': specs, 'author': author, 'link': link})

In [145]:
def retrieve_for_batch(batch):
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.get("https://smartify.org/")
    time.sleep(1)
    
    all_illus = glob.glob(batch + "/*.jpg")
    paths = ['documents', 'DFKV', 'DFKV-illustrations', '5_illustration_enrichment', batch, all_illus[0].split('\\')[-1].split('.')[0]]
    data = []
    # first illustration
    previous_url = driver.current_url
    button = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/header/div/section/div[2]/div[1]/div/div/button")
    button.click() # This opens the windows file selector
    time.sleep(1)
    
    for step in paths:
        time.sleep(1)
        pyautogui.write(step)
        pyautogui.press('enter')  
    time.sleep(1)
    
    if previous_url != driver.current_url:
        data.append(get_painting_info(illu_name))
            
    for illu in tqdm(all_illus[1:]):

        previous_url = driver.current_url

        illu_name = illu.split('\\')[-1].split('.')[0]
        button = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/header/div/section/div[2]/div[1]/div/div/button")
        button.click() # This opens the windows file selector
        time.sleep(1)
        pyautogui.write(illu_name) 
        pyautogui.press('enter')
        time.sleep(2)
        if previous_url != driver.current_url:
            data.append(get_painting_info(driver, illu_name))
            df = pd.DataFrame(data)
            df.to_csv(batch + '.csv') # intermediate saves

    df = pd.DataFrame(data)
    df.to_csv(batch + '.csv')
    return df

In [146]:
df = retrieve_for_batch('batch9')

C:\Users\emichelet\AppData\Local\Temp\ipykernel_13168\735552752.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')
100%|██████████| 454/454 [24:56<00:00,  3.30s/it]


In [142]:
df = pd.read_csv('batch9.csv')

In [143]:
df.tail()

,Unnamed: 0,id,title,specs,author,link
119,119,ILLU_12753_5003_2,\n Morning\n,\n Oil on canvas | 22 x 30.5cm | c....,\n Caspar David Friedrich\n,https://commons.wikimedia.org/wiki/File:CDFrie...
120,120,ILLU_13061_274_0,\n The Railway Bridge over Avenue M...,\n Oil on canvas | 92 x 73cm | 1888...,\n Vincent van Gogh\n,NaN
121,121,ILLU_13062_115_0,\n Polynesian Woman with Children\n...,\n Oil on linen canvas | 97 x 74cm ...,\n Paul Gauguin\n,NaN
122,122,ILLU_13065_224_0,\n The Burden (The Laundress)\n ...,\n Oil on canvas | 130 x 98cm | 185...,\n Honoré Daumier\n,NaN
123,123,ILLU_13083_238_0,\n Portrait of Gertrud Muller\n ...,\n \n,\n Ferdinand Hodler\n,NaN


In [144]:
all_illus = glob.glob("batch9/*.jpg")
all_illus.index("batch9\ILLU_13083_238_0.jpg")

599

In [102]:
df2.append(df).drop(columns = ['Unnamed: 0']).to_csv('batch3.csv')

In [103]:
df2 = pd.read_csv('batch3.csv')

In [104]:
df2

,Unnamed: 0,id,title,specs,author,link
0,0,ILLU_10332_144_0,\n Interior\n,\n Oil on canvas | 813.3 x 1143.8mm...,\n Edgar Degas\n,https://en.wikipedia.org/wiki/Interior_(Degas)
1,1,ILLU_10332_93_0,\n A Matador\n,\n Oil on canvas | 171.1 x 113cm | ...,\n Edouard Manet\n,https://www.metmuseum.org/art/collection/searc...
2,2,ILLU_10345_173_0,\n The Promenade\n,\n Oil on canvas | 81.3 x 64.8cm | ...,\n Pierre-Auguste Renoir\n,https://www.getty.edu/art/collection/objects/8...
3,3,ILLU_10346_253_0,\n Caricature de Delacroix (reprodu...,\n Procédé photomécanique\n,\n George Sand\n,https://parismuseescollections.paris.fr/node/2...
4,4,ILLU_10350_384_0,\n Sunflowers\n,\n 1888\n,\n Vincent van Gogh\n,NaN
...,...,...,...,...,...,...
154,154,ILLU_15697_260_0,\n The Spangled Cock\n,\n Watercolor heightened with gouac...,\n Joseph Crawhall\n,https://collections.britishart.yale.edu/catalo...
155,155,ILLU_15717_214_0,\n Hommage à Delacroix\n,\n Huile sur toile | 160 x 250cm | ...,\n Henri Fantin-Latour\n,https://commons.wikimedia.org/wiki/File:Hommag...
156,0,ILLU_16289_35_0,\n Full moon\n,\n 1919\n,\n Paul Klee\n,NaN
157,1,ILLU_16297_131_1,\n Old town II\n,\n Oil on canvas | 52 x 78.5cm | 19...,\n Vassily Kandinsky\n,NaN
